In [3]:
from pathlib import Path
import json
import pandas as pd
import numpy as np
from itertools import product
from chan import detect_zhongshu, analyze_zhongshu

ROOT = Path("/Users/cc/PycharmProjects/ChanLab")
SEG_DIRS = {
    "slow": ROOT / "chan_data" / "segments" / "slow",
    "fast": ROOT / "chan_data" / "segments" / "fast",
}
CFG = ROOT / "configs"
REPORT_DIR = ROOT / "reports"
REPORT_DIR.mkdir(parents=True, exist_ok=True)

SYMBOLS = ["BTCUSDT", "ETHUSDT"]
TFS = ["5m","15m","30m","1h","4h"]

BASE = dict(
    level="segment",
    require_alt_dir=None,
    min_hits=3,
    leave_confirm=1,
    allow_overlap_zs=True,
    enable_quality=True,
    enhancements=dict(
        strict_alternating_direction=True,
        min_zs_width_ratio=0.005,
        max_extension_units=20,
        confirm_third_point=True,
        tp_pullback_max_units=3,
        tp_continuation_max_units=3,
        tp_buffer_ratio=0.15,
        tp_use_atr_band=True,
        tp_atr_k=0.5,
        zs_adj_abs_tol=1e-6,
        zs_adj_rel_tol=0.0005,
        zs_adj_k_atr=0.25,
    )
)

ALT_DIR = [True, False]
MIN_HITS = [3, 4]
MIN_WIDTH = [0.003, 0.005, 0.008]
TP_CONFIRM = [True, False]
TP_PULLBACK_UNITS = [2, 3]
TP_CONT_UNITS = [2, 3]
TP_BUFFER = [0.10, 0.15]

def make_cfg(require_alt_dir, min_hits, min_width, tp_confirm, tp_pb, tp_ct, tp_buf):
    cfg = json.loads(json.dumps(BASE))
    cfg["require_alt_dir"] = require_alt_dir
    cfg["min_hits"] = int(min_hits)
    enh = cfg["enhancements"]
    enh["min_zs_width_ratio"] = float(min_width)
    enh["confirm_third_point"] = bool(tp_confirm)
    enh["tp_pullback_max_units"] = int(tp_pb)
    enh["tp_continuation_max_units"] = int(tp_ct)
    enh["tp_buffer_ratio"] = float(tp_buf)
    return cfg

def load_segments(version: str, symbol: str, tf: str) -> pd.DataFrame | None:
    base = SEG_DIRS.get(version)
    if not base:
        return None
    p = base / symbol / f"{tf}_segments.csv"
    if not p.exists():
        return None
    return pd.read_csv(p, parse_dates=["timestamp_start","timestamp_end"])

def summarize_one(seg_df: pd.DataFrame, cfg: dict) -> dict:
    zs = detect_zhongshu(seg_df, **cfg)
    st = analyze_zhongshu(zs)
    grade = st.get("grade_counts", {})
    return dict(
        total_zs=st.get("total_zs", 0),
        median_width_rel=st.get("median_width_rel", 0.0),
        median_duration_h=st.get("median_duration_h", 0.0),
        grade_A=int(grade.get("A", 0)),
        grade_B=int(grade.get("B", 0)),
        grade_C=int(grade.get("C", 0)),
    )

rows = []
cfg_id = 0

for version, base_dir in SEG_DIRS.items():
    if not base_dir.exists():
        print(f"[skip] segments version not found: {version} -> {base_dir}")
        continue

    for (alt, mh, mw, tp_c, tp_pb, tp_ct, tp_buf) in product(
        ALT_DIR, MIN_HITS, MIN_WIDTH, TP_CONFIRM, TP_PULLBACK_UNITS, TP_CONT_UNITS, TP_BUFFER
    ):
        cfg_id += 1
        cfg = make_cfg(alt, mh, mw, tp_c, tp_pb, tp_ct, tp_buf)

        for s in SYMBOLS:
            for tf in TFS:
                seg = load_segments(version, s, tf)
                if seg is None or seg.empty:
                    print(f"[skip] {version}/{s}/{tf} (missing or empty)")
                    continue
                try:
                    stat = summarize_one(seg, cfg)
                except Exception as e:
                    print(f"[error] {version}-{s}-{tf} cfg#{cfg_id}: {e}")
                    continue

                rows.append({
                    "version": version,
                    "cfg_id": cfg_id,
                    "symbol": s,
                    "tf": tf,
                    "alt_dir": alt,
                    "min_hits": mh,
                    "min_width": mw,
                    "tp_confirm": tp_c,
                    "tp_pb": tp_pb,
                    "tp_ct": tp_ct,
                    "tp_buf": tp_buf,
                })

if not rows:
    print("[done] There is no available data or all combinations are skipped. Please check whether segments_slow/segments_fast has been delivered.")
else:
    df = pd.DataFrame(rows)
    sort_cols = [c for c in ["version","symbol","tf","cfg_id"] if c in df.columns]
    df = df.sort_values(sort_cols).reset_index(drop=True)
    out_csv = REPORT_DIR / "zhongshu_grid_summary.csv"
    df.to_csv(out_csv, index=False)
    print(f"[done] Total evaluation {len(df)} result -> {out_csv}")

    def topk(group, by, k=3):
        cols = ["cfg_id","alt_dir","min_hits","min_width","tp_confirm","tp_pb","tp_ct","tp_buf",
                "total_zs","grade_A","grade_B","grade_C","median_width_rel","median_duration_h"]
        return group.sort_values(by=by, ascending=False).head(k)[cols]

    for s in SYMBOLS:
        for tf in TFS:
            sub = df[(df.symbol==s) & (df.tf==tf)]
            if sub.empty:
                continue
            print(f"\n=== {s}-{tf} | version=slow ===")
            slow_sub = sub[sub.version=="slow"]
            if not slow_sub.empty:
                print("Top by total_zs:")
                display(topk(slow_sub, "total_zs"))
                print("Top by grade_A+grade_B:")
                slow_sub2 = slow_sub.assign(AB=lambda x: x["grade_A"]+x["grade_B"])
                display(topk(slow_sub2, "AB"))
            print(f"\n=== {s}-{tf} | version=fast ===")
            fast_sub = sub[sub.version=="fast"]
            if not fast_sub.empty:
                print("Top by total_zs:")
                display(topk(fast_sub, "total_zs"))
                print("Top by grade_A+grade_B:")
                fast_sub2 = fast_sub.assign(AB=lambda x: x["grade_A"]+x["grade_B"])
                display(topk(fast_sub2, "AB"))


[done] 一共评估 3840 行结果 -> /Users/cc/PycharmProjects/ChanLab/reports/zhongshu_grid_summary.csv

=== BTCUSDT-5m | version=slow ===
Top by total_zs:


cfg_id  alt_dir  min_hits  min_width  tp_confirm  tp_pb  tp_ct  tp_buf  \
2799     112    False         3      0.003       False      3      3    0.15   
2798     111    False         3      0.003       False      3      3    0.10   
2797     110    False         3      0.003       False      3      2    0.15   

      total_zs  grade_A  grade_B  grade_C  median_width_rel  median_duration_h  
2799        12        0        3        9          0.016448         251.083333  
2798        12        0        3        9          0.016448         251.083333  
2797        12        0        3        9          0.016448         251.083333

Top by grade_A+grade_B:


cfg_id  alt_dir  min_hits  min_width  tp_confirm  tp_pb  tp_ct  tp_buf  \
2718      31     True         3      0.005       False      3      3    0.10   
2719      32     True         3      0.005       False      3      3    0.15   
2734      47     True         3      0.008       False      3      3    0.10   

      total_zs  grade_A  grade_B  grade_C  median_width_rel  median_duration_h  
2718         4        2        2        0          0.019832         251.083333  
2719         4        2        2        0          0.019832         251.083333  
2734         4        2        2        0          0.019832         251.083333


=== BTCUSDT-5m | version=fast ===
Top by total_zs:


...

cfg_id  alt_dir  min_hits  min_width  tp_confirm  tp_pb  tp_ct  tp_buf  \
3552      97    False         3      0.003        True      2      2    0.10   
3587     132    False         3      0.008        True      2      3    0.15   
3578     123    False         3      0.005       False      2      3    0.10   

      total_zs  grade_A  grade_B  grade_C  median_width_rel  median_duration_h  
3552         1        1        0        0          0.102162             1224.0  
3587         1        1        0        0          0.102162             1224.0  
3578         1        1        0        0          0.102162             1224.0


=== ETHUSDT-4h | version=fast ===
Top by total_zs:


cfg_id  alt_dir  min_hits  min_width  tp_confirm  tp_pb  tp_ct  tp_buf  \
1663     320    False         3      0.005       False      3      3    0.15   
1644     301    False         3      0.003       False      3      2    0.10   
1679     336    False         3      0.008       False      3      3    0.15   

      total_zs  grade_A  grade_B  grade_C  median_width_rel  median_duration_h  
1663         4        1        0        3          0.056613             1170.0  
1644         4        1        0        3          0.056613             1170.0  
1679         4        1        0        3          0.056613             1170.0

Top by grade_A+grade_B:


cfg_id  alt_dir  min_hits  min_width  tp_confirm  tp_pb  tp_ct  tp_buf  \
1577     234     True         3      0.008       False      2      2    0.15   
1548     205     True         3      0.003       False      3      2    0.10   
1561     218     True         3      0.005       False      2      2    0.15   

      total_zs  grade_A  grade_B  grade_C  median_width_rel  median_duration_h  
1577         3        1        1        1          0.064842              748.0  
1548         3        1        1        1          0.064842              748.0  
1561         3        1        1        1          0.064842              748.0

In [5]:

from pathlib import Path
import json
import pandas as pd
from chan import detect_zhongshu, validate_chan_zhongshu

ROOT = Path("/Users/cc/PycharmProjects/ChanLab")
SEG = {
    "slow": ROOT / "chan_data" / "segments" / "slow",
    "fast": ROOT / "chan_data" / "segments" / "fast",
}
OUT = {
    "slow": ROOT / "chan_data" / "zhongshu_slow",
    "fast": ROOT / "chan_data" / "zhongshu_fast"
}
CFG = {
    "slow": json.load(open(ROOT/"configs/zhongshu_slow_default.json","r")),
    "fast": json.load(open(ROOT/"configs/zhongshu_fast_default.json","r"))
}

for v in OUT.values():
    v.mkdir(parents=True, exist_ok=True)

SYMBOLS = ["BTCUSDT","ETHUSDT"]
TFS = ["5m","15m","30m","1h","4h"]

def run_and_save(version, symbol, tf):
    seg_dir = SEG[version] / symbol
    p = seg_dir / f"{tf}_segments.csv"
    if not p.exists():
        print(f"[skip] {version}/{symbol}/{tf} (no segments csv)")
        return
    df = pd.read_csv(p, parse_dates=["timestamp_start","timestamp_end"])
    if df.empty:
        print(f"[skip] {version}/{symbol}/{tf} (empty)")
        return

    zs = detect_zhongshu(df, **CFG[version])
    rep = validate_chan_zhongshu(zs, f"{version}-{symbol}-{tf}", verbose=False)

    out_dir = OUT[version] / symbol
    out_dir.mkdir(parents=True, exist_ok=True)
    out = out_dir / f"{tf}_zhongshu.csv"
    zs.to_csv(out, index=False)

    print(f"[saved] {version}-{symbol}-{tf}: zs={len(zs)} -> {out}")
    print(f"        grade_dist={rep.get('grade_dist',{})} relation={rep.get('relation_dist',{})}")

for v in ["slow","fast"]:
    if not SEG[v].exists():
        print(f"[warn] segments path missing: {SEG[v]}")
        continue
    for s in SYMBOLS:
        for tf in TFS:
            run_and_save(v, s, tf)


[saved] slow-BTCUSDT-5m: zs=4 -> /Users/cc/PycharmProjects/ChanLab/chan_data/zhongshu_slow/BTCUSDT/5m_zhongshu.csv
        grade_dist={'A': 2, 'C': 1, 'B': 1} relation={'separate': 2, 'root': 1, 'expansion': 1}
[saved] slow-BTCUSDT-15m: zs=5 -> /Users/cc/PycharmProjects/ChanLab/chan_data/zhongshu_slow/BTCUSDT/15m_zhongshu.csv
        grade_dist={'B': 3, 'C': 2} relation={'separate': 2, 'root': 1, 'adjacent': 1, 'expansion': 1}
[saved] slow-BTCUSDT-30m: zs=2 -> /Users/cc/PycharmProjects/ChanLab/chan_data/zhongshu_slow/BTCUSDT/30m_zhongshu.csv
        grade_dist={'C': 1, 'B': 1} relation={'root': 1, 'separate': 1}
[saved] slow-BTCUSDT-1h: zs=1 -> /Users/cc/PycharmProjects/ChanLab/chan_data/zhongshu_slow/BTCUSDT/1h_zhongshu.csv
        grade_dist={'A': 1} relation={'root': 1}
[saved] slow-BTCUSDT-4h: zs=0 -> /Users/cc/PycharmProjects/ChanLab/chan_data/zhongshu_slow/BTCUSDT/4h_zhongshu.csv
        grade_dist={} relation={}
[saved] slow-ETHUSDT-5m: zs=15 -> /Users/cc/PycharmProjects/ChanLab